In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# ========================= 初始化頻率範圍與變數 ========================= #
omega = np.linspace(-np.pi, np.pi, 16000)  # 頻率範圍 [-π, π]，分成 16000 點
z = np.exp(1j * omega)  # 定義複數域單位圓 z = e^(jω)

# ========================= H1(z) 計算（對數域） ========================= #
H1_numerator = (1 - 0.98 * np.exp(1j * 0.8 * np.pi) / z) * (1 - 0.98 * np.exp(-1j * 0.8 * np.pi) / z)  # 零點部分
H1_denominator = (1 - 0.8 * np.exp(1j * 0.4 * np.pi) / z) * (1 - 0.8 * np.exp(-1j * 0.4 * np.pi) / z)  # 極點部分
log_H1 = np.log(H1_numerator) - np.log(H1_denominator)  # H1(z) 轉換為對數域

# ========================= H2(z) 計算（變化部分，對數域） ========================= #
k_values = [1, 2, 3, 4]  # k 值列表
log_H2 = np.zeros_like(omega, dtype=np.complex128)  # 初始化 H2 的對數域結果為 0

for k in k_values:
    ck = 0.95 * np.exp(1j * (0.15 * np.pi + 0.02 * np.pi * k))  # 計算零點位置 c_k
    term = (((np.conj(ck) - 1 / z) * (ck - 1 / z)) /
            ((1 - ck / z) * (1 - np.conj(ck) / z))) ** 2  # 傳遞函數部分計算
    log_H2 += np.log(term)  # 累加對數域結果

# ========================= 合併 H1 和 H2，計算頻率響應 ========================= #
log_H = log_H1 + log_H2  # 將 H1 和 H2 合併
H = np.exp(log_H)  # 從對數域轉回線性域

# 計算主值相位與展開相位
principal_phase = np.angle(H)  # 主值相位 [-π, π]
continuous_phase = np.unwrap(principal_phase) + 50  # 展開相位，並上移 50

# 計算組時延（Group Delay）
frequencies = np.linspace(-np.pi, np.pi, len(H))  # 頻率網格
group_delay_values = -np.gradient(np.unwrap(np.angle(H)), frequencies)  # 組時延計算

# ========================= 圖 1: 主值相位與展開相位 ========================= #
plt.figure(figsize=(10, 8))

# (a) 主值相位
plt.subplot(2, 1, 1)
plt.plot(omega, principal_phase, label="Principal Phase")
plt.xlabel("Frequency (rad/sample)")
plt.ylabel(r"$ARG[H(e^{j\omega})]$")
plt.xticks(ticks=np.linspace(-np.pi, np.pi, 11),
           labels=[f"{t/np.pi:.1g}$\pi$" if t not in [0, -np.pi, np.pi] else
                   (r"$-\pi$" if t == -np.pi else r"$\pi$" if t == np.pi else "0")
                   for t in np.linspace(-np.pi, np.pi, 11)])
plt.yticks(np.arange(-4, 5, 2))
plt.grid()
plt.title("(a) Principal Value of Phase Response")

# (b) 展開相位
plt.subplot(2, 1, 2)
plt.plot(omega, continuous_phase, label="Unwrapped Phase")
plt.xlabel("Frequency (rad/sample)")
plt.ylabel(r"$arg[H(e^{j\omega})]$")
plt.yticks(np.arange(-60, 61, 20))
plt.grid()
plt.title("(b) Unwrapped Phase Response")

plt.tight_layout()
plt.show()

# ========================= 圖 2: 組時延與幅度響應 ========================= #
plt.figure(figsize=(10, 8))

# (a) 組時延
plt.subplot(2, 1, 1)
plt.plot(omega / np.pi, group_delay_values)
plt.xlabel(r"$\omega$")
plt.ylabel(r"$\mathrm{grd}[H(e^{j\omega})]$")
plt.xticks(ticks=np.linspace(-1, 1, 11), labels=[f"{x:.1g}$\pi$" for x in np.linspace(-1, 1, 11)])
plt.yticks(np.arange(-50, 201, 50))
plt.grid()
plt.title("(a) Group Delay of $H(z)$")

# (b) 幅度響應
plt.subplot(2, 1, 2)
magnitude = np.abs(H) / (np.max(np.abs(H)) / 2.2)  # 正規化幅度
plt.plot(omega / np.pi, magnitude)
plt.xlabel(r"$\omega$")
plt.ylabel(r"$|H(e^{j\omega})|$")
plt.xticks(ticks=np.linspace(-1, 1, 11), labels=[f"{x:.1g}$\pi$" for x in np.linspace(-1, 1, 11)])
plt.yticks(np.arange(0, 2.6, 0.5))
plt.grid()
plt.title("(b) Magnitude of Frequency Response")

plt.tight_layout()
plt.show()

# ========================= 圖 3: 訊號波形與 DTFT ========================= #
plt.figure(figsize=(10, 8))

# 生成分段訊號 x[n]
N = 300  # 訊號長度
n = np.arange(N)
x_n = np.zeros(N)
x_n[0:63] = np.sin(0.8 * np.pi * n[:63]) * np.hanning(63)  # 第一段：頻率 0.8π
x_n[63:125] = np.sin(0.2 * np.pi * (n[63:125] - 63)) * np.hanning(62)  # 第二段：頻率 0.2π
x_n[125:188] = np.sin(0.4 * np.pi * (n[125:188] - 125)) * np.hanning(63)  # 第三段：頻率 0.4π

# 計算 DTFT
omega = np.linspace(-np.pi, np.pi, 1024)
X = np.array([np.sum(x_n * np.exp(-1j * w * n)) for w in omega])

# (a) 訊號波形
plt.subplot(2, 1, 1)
plt.plot(n, x_n)
plt.title("(a) Waveform of signal x[n]")
plt.xlabel("Sample number (n)")
plt.ylabel("Amplitude")
plt.yticks(np.arange(-1, 1.5, 0.5))  # 修改 Y 座標從 -1 到 1，0.5 為一格
plt.grid()

# (b) DTFT 幅度響應
plt.subplot(2, 1, 2)
plt.plot(omega / np.pi, np.abs(X))
plt.title("(b) Magnitude of DTFT of $x[n]$")
plt.xlabel("$\omega$ (Normalized Frequency)")
plt.ylabel(r"$|X(e^{j\omega})|$")
plt.yticks(np.arange(0, 21, 5))  # 修改 Y 座標從 0 到 20，5 為一格
plt.grid()

plt.tight_layout()
plt.show()
